# NFLX Data Analysis

In [4]:
import duckdb
import pandas as pd

In [77]:
df_top10 = pd.read_excel('Data Specialist NFLX Data.xlsx',sheet_name='NFLX Top 10')
df_imdb = pd.read_excel('Data Specialist NFLX Data.xlsx',sheet_name='IMDB Rating')
df_runtime = pd.read_excel('Data Specialist NFLX Data.xlsx',sheet_name='Runtime')



### EDA

In [78]:
df_top10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_added                  520 non-null    datetime64[ns]
 1   week                        520 non-null    datetime64[ns]
 2   category                    520 non-null    object        
 3   show_title                  520 non-null    object        
 4   season_title                249 non-null    object        
 5   weekly_rank                 520 non-null    int64         
 6   cumulative_weeks_in_top_10  520 non-null    int64         
 7   weekly_hours_viewed         520 non-null    int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 32.6+ KB


In [79]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15879 entries, 0 to 15878
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   14569 non-null  object 
 1   rating  15879 non-null  float64
dtypes: float64(1), object(1)
memory usage: 248.2+ KB


In [80]:
df_runtime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15879 entries, 0 to 15878
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    14569 non-null  object
 1   runtime  15879 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 248.2+ KB


In [81]:
df_runtime= df_runtime.dropna(axis=0)

In [82]:
df_runtime['title'].value_counts()

title
Life                       5
The Gift                   4
The Bodyguard              3
Death Note                 3
Tiger                      3
                          ..
My Liberation Notes        1
Radhe Shyam (Hindi)        1
Rambo: Last Blood          1
Raw (Hindi)                1
Retfærdighedens ryttere    1
Name: count, Length: 14226, dtype: int64

In [83]:
df_runtime = df_runtime.drop_duplicates(subset='title')

The system we use to collect data from the Netflix website experienced an issue during the
 week of May22nd, 2022, specifically affecting the 'weekly_hours_viewed' column. All other
 columns were accurately collected. Due to this, we cannot use this week in our estimates, as the
 'weekly_hours_viewed' column is critical for tracking viewership metrics. You can assume that
 this is the only week with incomplete viewership data

In [18]:
#dropping rows when the week is on 2022/05/22
df_top10 = df_top10[df_top10['week'] != '2022-05-22']

In [17]:


query = '''select distinct week from df_top10
'''

duckdb.sql(query)


┌─────────────────────┐
│        week         │
│    timestamp_ns     │
├─────────────────────┤
│ 2022-05-08 00:00:00 │
│ 2022-06-26 00:00:00 │
│ 2022-04-10 00:00:00 │
│ 2022-06-19 00:00:00 │
│ 2022-04-24 00:00:00 │
│ 2022-04-17 00:00:00 │
│ 2022-06-12 00:00:00 │
│ 2022-05-01 00:00:00 │
│ 2022-04-03 00:00:00 │
│ 2022-05-15 00:00:00 │
│ 2022-05-29 00:00:00 │
│ 2022-06-05 00:00:00 │
├─────────────────────┤
│       12 rows       │
└─────────────────────┘

### Analysis and answers

1. In the ‘Films (English)’ category, which film has the most appearances in our data set
 (NFLX Top 10 tab of the Sheet)? What were its average weekly hours viewed?

In [101]:


query = '''
select
  show_title
 ,count(show_title) as title_frequency
 ,avg(weekly_hours_viewed) as average_views
 from df_top10
 where category = 'Films (English)'
 group by show_title
 order by 2 desc
 limit 1
'''

duckdb.sql(query)


┌────────────────────┬─────────────────┬───────────────┐
│     show_title     │ title_frequency │ average_views │
│      varchar       │      int64      │    double     │
├────────────────────┼─────────────────┼───────────────┤
│ Sonic the Hedgehog │               8 │     7481250.0 │
└────────────────────┴─────────────────┴───────────────┘

2. In the 'Films (English)' category, which film has the lowest IMDb rating? What were its
 average weekly hours viewed?

In [102]:

query = '''
select
  a.show_title
 ,a.category
 ,avg(a.weekly_hours_viewed) as average_views
 ,b.rating
from df_top10 a
 join df_imdb b
  on a.show_title = b.title
where a.category = 'Films (English)' and b.rating > 0
group by a.show_title, b.rating, a.category
order by b.rating asc
limit 1
'''

duckdb.sql(query)


┌────────────────────┬─────────────────┬───────────────┬────────┐
│     show_title     │    category     │ average_views │ rating │
│      varchar       │     varchar     │    double     │ double │
├────────────────────┼─────────────────┼───────────────┼────────┤
│ 365 Days: This Day │ Films (English) │    29022500.0 │    2.5 │
└────────────────────┴─────────────────┴───────────────┴────────┘

3. In the 'Films (Non-English)' category, which film has spent the most weeks in the top 10?
 To estimate the number of users who watched this film, what assumptions would you
 make andwhat risks are involved?

In [91]:
query = '''
select
     show_title
    ,cumulative_weeks_in_top_10
    from df_top10
    where category = 'Films (Non-English)'
    order by 2 desc
    limit 1
'''

duckdb.sql(query)

┌───────────────────┬────────────────────────────┐
│    show_title     │ cumulative_weeks_in_top_10 │
│      varchar      │           int64            │
├───────────────────┼────────────────────────────┤
│ Through My Window │                         13 │
└───────────────────┴────────────────────────────┘

In [97]:



query = '''
select
     a.show_title
    ,a.total_minutes_viewed
    ,b.runtime
    ,floor(a.total_minutes_viewed / b.runtime) as total_watched
from
(
select
     show_title
    ,cumulative_weeks_in_top_10
    ,sum(weekly_hours_viewed) * 60 as total_minutes_viewed
    from df_top10
    where category = 'Films (Non-English)'
    group by show_title, cumulative_weeks_in_top_10
    order by 2 desc
    limit 1
) a
join df_runtime b
on a.show_title = b.title


'''
duckdb.sql(query)

┌───────────────────┬──────────────────────┬─────────┬───────────────┐
│    show_title     │ total_minutes_viewed │ runtime │ total_watched │
│      varchar      │        int128        │  int64  │    double     │
├───────────────────┼──────────────────────┼─────────┼───────────────┤
│ Through My Window │             90000000 │     116 │      775862.0 │
└───────────────────┴──────────────────────┴─────────┴───────────────┘

In [95]:

#this film has been watched 775862 times. Assuming it was watched once by every single user, it had 775862 users.

 4 .Whatarethe risks of ignoring the data from the week of May 22nd when calculating the
 metrics from the previous questions?

In [98]:
#The week was removed due to the issue with the weekly_hours_viewed field. This removal could affect the analysis if a film during this week was on the list of most-watched films, or if a film from this week had more weeks in the top 10

5. While we've indicated that the 'weekly_hours_viewed' data for the week of May 22nd
 cannot be used in our estimates, we may want to fill in this missing information for other
 analyses. As a Data Specialist, what methodology would you propose to estimate the
 'weekly_hours_viewed' for this missing week?

In [100]:
#One of the first approaches would be to examine the
# behavior of the weekly_hours_viewed data before and
# after the missing week. Using the mean or median of adjacent weeks,
# I could generate an estimate for the week of May 22nd.